<a href="https://colab.research.google.com/github/SeekerHub/Twitter_Sentiment_Analysis/blob/master/Sentiment_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_data = pd.read_csv('/content/train_2kmZucJ.csv',)
train_data = train_data.reset_index(drop = True)
test_data = pd.read_csv('/content/test_oJQbWVk.csv')
test_data = test_data.reset_index(drop = True)

In [3]:
train_data.shape

(7920, 3)

In [4]:
test_data.shape

(1953, 2)

In [5]:
train_data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [7]:
test_data.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [8]:
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Removing Emoticons**


*   Tweet nowadays are mostly field with emoticons which are unecessary for our analysis.
*   Handling both emoticons and unnecessary punctuations.



In [24]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3' , '^^', '^'
    ])

#Sad emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

emoticons = emoticons_happy.union(emoticons_sad)

!pip install tweet-preprocessor

In [25]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [26]:
def clean_text_1(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
    #after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []#looping through conditions
    for w in word_tokens:
      #check tokens against stop words , emoticons and punctuations
      if w not in stop_words and w not in emoticons and w not in string.punctuation:
        filtered_tweet.append(w)
    return ' '.join(filtered_tweet)

round_1 = lambda x: clean_text_1(x)   

In [27]:
def remove_urls(text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    # text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
    return text

round_2 = lambda x: remove_urls(x) 

def clean_text_2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    # text = re.sub('[^]', '', text)
    # text = re.sub('\n', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('[^a-zA-Z]', '', text)
    
    return text

round_3 = lambda x: clean_text_2(x)

In [28]:
data_clean = pd.DataFrame(train_data.tweet.apply(round_2))
data_clean = pd.DataFrame(data_clean.tweet.apply(round_1))


data_clean.tweet[1]

'Finally transparant silicon case Thanks uncle yay Sony Xperia S sonyexperias…'

In [ ]:
# data_clean = pd.DataFrame(data_clean.tweet.apply(round_3))

# data_clean.tweet[1]

'FinallytransparantsiliconcaseThanksuncleyaySonyXperiaSsonyexperias'

In [29]:
# ry = 'the king ^^ so as know'
# text = ry.split()
# text = re.sub('[^a-zA-Z]', '', text)
# text

In [30]:
def Stemming(text):
  corpus = []
  text = text.lower()
  text = text.split()
  """Applying Steming"""
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  ps = PorterStemmer() #loved --> love word thast essentially means the same
  text = [ps.stem(word) for word in text if not word in set(all_stopwords)]
  corpus = ' '.join(text)
  return corpus

stemming = lambda x: Stemming(x)


In [34]:
train_data_clean = pd.DataFrame(data_clean.tweet.apply(stemming))


In [37]:
import pickle
# Let's pickle it for later use
train_data_clean.to_pickle("corpus.pkl")